In [1]:
# Required libraries
import matplotlib.pyplot as plt
import networkx as nx
import heapq
import random
import time
from statistics import mean 

In [2]:
#generate graph


def graph_gen(V, density):
    # Generates a connected graph with V nodes and target density.
    G = nx.Graph()
    G.add_nodes_from(range(V))

    # Calculate the number of edges base on density
    edges_num = int(density * V * (V - 1) / 2)

    # Ensure minimum number of edges for connectivity
    edges_num = max(edges_num, V-1)

    # Generate edges randomly
    edges = set()
    while len(edges) < edges_num:
        u = random.randint(0, V - 1)
        v = random.randint(0, V - 1)
        if u != v and (u, v) not in edges and (v, u) not in edges:
            edges.add((u, v))

    # Add edges to the graph
    G.add_edges_from(edges)

    # Random the weight of edges
    for (u, v) in G.edges():
        G.edges[u, v]['weight'] = random.randint(1, 25)
    # Ensure connectivity
    if not nx.is_connected(G):
        return graph_gen(V, density)

    return G
        
def neg_graph_gen(V, density):
    #Generates a connected graph with V nodes and target density with negative weight.
    G = nx.Graph()
    G.add_nodes_from(range(V))

    # Calculate the number of edges base on density
    edges_num = int(density * V * (V - 1) / 2)

    # Ensure minimum number of edges for connectivity
    edges_num = max(edges_num, V-1)

    # Generate edges randomly
    edges = set()
    while len(edges) < edges_num:
        u = random.randint(0, V - 1)
        v = random.randint(0, V - 1)
        if u != v and (u, v) not in edges and (v, u) not in edges:
            edges.add((u, v))

    # Add edges to the graph
    G.add_edges_from(edges)

    # Random the weight of edges
    for (u, v) in G.edges():
        weight_check = random.randint(-25, 24)
        if weight_check >= 0:
            G.edges[u, v]['weight'] = weight_check + 1
        else:
            G.edges[u, v]['weight'] = weight_check

        
    # Ensure connectivity
    if not nx.is_connected(G):
        return neg_graph_gen(V, density)
    return G

#Prim algorithm with time calculation
def prim_mst(graph):
    """Finds the Minimum Spanning Tree of a weighted graph using Prim's algorithm."""
    
    start_time = time.perf_counter()
    
    # Graph for MST and total weight of it
    MST = nx.Graph()
    total_weight = 0
    # Start with an optional node, here 0
    start = list(graph.nodes())[0]
    # Set to store visited nodes
    visited = set()
    visited.add(start)
    # Priority queue to store edges with their weight
    edge_queue = []
    # Add all edges, neighbors of starting node to queue
    for neighbor, values in graph[start].items():
        heapq.heappush(edge_queue, (values['weight'], start, neighbor))
    
    # Go for edges with lowest weight first
    while edge_queue is not None and len(MST.nodes) < len(graph.nodes):
        weight, node, neighbor = heapq.heappop(edge_queue)

        if neighbor not in visited:
            visited.add(neighbor)
            MST.add_edge(node, neighbor, weight = weight)
            total_weight += weight

            # Add all edges, neighbors of new nodes to queue
            for neighbor_v, values in graph[neighbor].items():
                if neighbor_v not in visited:
                    heapq.heappush(edge_queue, (values['weight'], neighbor, neighbor_v))
    
    run_time = (time.perf_counter() - start_time) * 1000  # Convert to milliseconds
    
    return MST, total_weight, run_time

#Kruskal algorithm with time calculation
def kruskal_mst(graph):
    """Finds the Minimum Spanning Tree of a weighted graph using Kruskal's algorithm."""

    start_time = time.perf_counter()

    def look(parent, i):
        if parent[i] == i:
            return i
        return look(parent, parent[i])

    def apply_union(parent, rank, x, y):
        xroot = look(parent, x)
        yroot = look(parent, y)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

    # Graph for MST and total weight of it
    MST = nx.Graph()
    total_weight = 0
    # Sort edges by their weight
    edge_list = sorted(graph.edges(data=True), key=lambda x: x[2]['weight'])

    parent = []
    rank = []

    for node in range(len(graph.nodes())):
        parent.append(node)
        rank.append(0)
    
    while len(MST.nodes()) < len(graph.nodes()):
        for u, v, values in edge_list:
            if look(parent, u) != look(parent, v):
                MST.add_edge(u, v, weight=values['weight'])
                apply_union(parent, rank, u, v)
    weight_list = list(MST.edges.data("weight", default=1))
    for i in range(len(weight_list)):
        total_weight += weight_list[i][2]

    run_time = (time.perf_counter() - start_time) * 1000  # Convert to milliseconds

    return MST, total_weight, run_time

In [ ]:
density_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
avg_time_prim = []
avg_time_kruskal = []